# Medicaid Data Cleaning and Feature Engineering

Data Source: https://data.medicaid.gov/State-Drug-Utilization/State-Drug-Utilization-Data-2016/3v6v-qk5s

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

In [2]:
# setting styles for plotting
plt.rcParams['figure.figsize'] = [20, 5]
plt.rcParams['image.cmap'] = 'bone'
sns.set_palette('bone')
sns.set_style('whitegrid')
sns.set_context('poster')

In [3]:
# read in 2016 medicaid state drug utilization data
drugs = pd.read_csv('sdued2016.csv')
print(drugs.shape)
drugs.head()

(4367625, 20)


,utilizationtype,state,labelercode,productcode,packagesize,year,quarter,productname,suppressionused,unitsreimbursed,numberofprescriptions,totalamountreimbursed,medicaidamountreimbursed,nonmedicaidamountreimbursed,quarterbegin,quarterbegindate,latitude,longitude,location,ndc
0,FFSU,AK,2,1433,80,2016,1,TRULICITY,True,NaN,NaN,NaN,NaN,NaN,1/1,01/01/2016,61.385,-152.2683,"(61.385, -152.2683)",2143380
1,FFSU,AK,2,1434,80,2016,1,TRULICITY,False,32.0,16.0,8882.87,8882.87,0.00,1/1,01/01/2016,61.385,-152.2683,"(61.385, -152.2683)",2143480
2,FFSU,AK,2,3227,30,2016,1,STRATTERA,False,1333.0,40.0,14311.75,13192.79,1118.96,1/1,01/01/2016,61.385,-152.2683,"(61.385, -152.2683)",2322730
3,FFSU,AK,2,3228,30,2016,1,STRATTERA,False,3175.0,93.0,34794.80,32385.37,2409.43,1/1,01/01/2016,61.385,-152.2683,"(61.385, -152.2683)",2322830
4,FFSU,AK,2,3229,30,2016,1,STRATTERA,False,3909.0,122.0,45969.37,42022.86,3946.51,1/1,01/01/2016,61.385,-152.2683,"(61.385, -152.2683)",2322930


In [4]:
drugs.columns

Index(['utilizationtype', 'state', 'labelercode', 'productcode', 'packagesize',
       'year', 'quarter', 'productname', 'suppressionused', 'unitsreimbursed',
       'numberofprescriptions', 'totalamountreimbursed',
       'medicaidamountreimbursed', 'nonmedicaidamountreimbursed',
       'quarterbegin', 'quarterbegindate', 'latitude', 'longitude', 'location',
       'ndc'],
      dtype='object')

In [5]:
# checking for opioids
drugs.loc[drugs.productname.str.contains('HYDROM', na=False)].sample(5)

,utilizationtype,state,labelercode,productcode,packagesize,year,quarter,productname,suppressionused,unitsreimbursed,numberofprescriptions,totalamountreimbursed,medicaidamountreimbursed,nonmedicaidamountreimbursed,quarterbegin,quarterbegindate,latitude,longitude,location,ndc
815612,MCOU,GA,409,3365,1,2016,1,HYDROMORPH,False,152.0,51.0,196.11,195.56,0.55,1/1,01/01/2016,32.9866,-83.6487,"(32.9866, -83.6487)",409336501
4134941,FFSU,WA,409,1283,5,2016,4,HYDROMORPH,False,116.0,21.0,350.55,350.55,0.00,10/1,10/01/2016,47.3917,-121.5708,"(47.3917, -121.5708)",409128305
1317997,FFSU,IN,406,3244,1,2016,4,HYDROMORPH,False,2398.0,19.0,315.24,276.24,39.00,10/1,10/01/2016,39.8647,-86.2604,"(39.8647, -86.2604)",406324401
42532,FFSU,AL,527,1354,1,2016,1,HYDROMORPH,False,1226.0,12.0,279.56,223.08,56.48,1/1,01/01/2016,32.7990,-86.8073,"(32.799, -86.8073)",527135401
1220272,MCOU,IN,54,265,25,2016,1,HYDROMORPH,False,1800.0,14.0,1021.10,1017.10,4.00,1/1,01/01/2016,39.8647,-86.2604,"(39.8647, -86.2604)",54026525


In [6]:
## testing to see if we have brand and generic named drugs...whch we do
drugs.loc[drugs.productname.str.contains('VICOD', na=False)].sample(5)

,utilizationtype,state,labelercode,productcode,packagesize,year,quarter,productname,suppressionused,unitsreimbursed,numberofprescriptions,totalamountreimbursed,medicaidamountreimbursed,nonmedicaidamountreimbursed,quarterbegin,quarterbegindate,latitude,longitude,location,ndc
2907329,FFSU,NY,74,3043,53,2016,2,VICODIN TA,True,NaN,NaN,NaN,NaN,NaN,4/1,04/01/2016,42.1497,-74.9384,"(42.1497, -74.9384)",74304353
2037969,MCOU,MN,74,3054,53,2016,2,VICODIN HP,True,NaN,NaN,NaN,NaN,NaN,4/1,04/01/2016,45.7326,-93.9196,"(45.7326, -93.9196)",74305453
1220867,MCOU,IN,74,3054,13,2016,1,VICODIN TA,True,NaN,NaN,NaN,NaN,NaN,1/1,01/01/2016,39.8647,-86.2604,"(39.8647, -86.2604)",74305413
3946877,MCOU,VA,74,3041,13,2016,3,VICODIN TA,False,1033.0,37.0,1248.65,1248.65,0.00,7/1,07/01/2016,37.7680,-78.2057,"(37.768, -78.2057)",74304113
2947141,FFSU,NY,74,3041,13,2016,3,VICODIN TA,False,710.0,19.0,837.85,664.01,173.84,7/1,07/01/2016,42.1497,-74.9384,"(42.1497, -74.9384)",74304113


In [7]:
# replacing NaNs with 0
drugs = drugs.replace(np.nan, 0)
drugs.head()

,utilizationtype,state,labelercode,productcode,packagesize,year,quarter,productname,suppressionused,unitsreimbursed,numberofprescriptions,totalamountreimbursed,medicaidamountreimbursed,nonmedicaidamountreimbursed,quarterbegin,quarterbegindate,latitude,longitude,location,ndc
0,FFSU,AK,2,1433,80,2016,1,TRULICITY,True,0.0,0.0,0.00,0.00,0.00,1/1,01/01/2016,61.385,-152.2683,"(61.385, -152.2683)",2143380
1,FFSU,AK,2,1434,80,2016,1,TRULICITY,False,32.0,16.0,8882.87,8882.87,0.00,1/1,01/01/2016,61.385,-152.2683,"(61.385, -152.2683)",2143480
2,FFSU,AK,2,3227,30,2016,1,STRATTERA,False,1333.0,40.0,14311.75,13192.79,1118.96,1/1,01/01/2016,61.385,-152.2683,"(61.385, -152.2683)",2322730
3,FFSU,AK,2,3228,30,2016,1,STRATTERA,False,3175.0,93.0,34794.80,32385.37,2409.43,1/1,01/01/2016,61.385,-152.2683,"(61.385, -152.2683)",2322830
4,FFSU,AK,2,3229,30,2016,1,STRATTERA,False,3909.0,122.0,45969.37,42022.86,3946.51,1/1,01/01/2016,61.385,-152.2683,"(61.385, -152.2683)",2322930


In [8]:
# group by state and drug and get sums
drugs2 = drugs.groupby(['state', 'productname'])['unitsreimbursed'].sum()
drugs2 = pd.DataFrame(data=drugs2).reset_index()
drugs2.head()

,state,productname,unitsreimbursed
0,AK,A-METHAPRE,0.0
1,AK,ABACAVIR,2024.0
2,AK,ABACAVIR-L,0.0
3,AK,ABILIFY,9579.0
4,AK,ABILIFY MA,98.0


In [9]:
drugs2['numberofprescriptions'] = drugs.groupby(['state', 'productname'])['numberofprescriptions'].sum().ravel()
drugs2['totalamountreimbursed'] = drugs.groupby(['state', 'productname'])['totalamountreimbursed'].sum().ravel()
drugs2['medicaidamountreimbursed'] = drugs.groupby(['state', 'productname'])['medicaidamountreimbursed'].sum().ravel()
drugs2['nonmedicaidamountreimbursed'] = drugs.groupby(['state', 'productname'])['nonmedicaidamountreimbursed'].sum().ravel()
drugs2.head()

,state,productname,unitsreimbursed,numberofprescriptions,totalamountreimbursed,medicaidamountreimbursed,nonmedicaidamountreimbursed
0,AK,A-METHAPRE,0.0,0.0,0.00,0.00,0.00
1,AK,ABACAVIR,2024.0,43.0,6736.14,6736.14,0.00
2,AK,ABACAVIR-L,0.0,0.0,0.00,0.00,0.00
3,AK,ABILIFY,9579.0,383.0,121183.60,120565.62,617.98
4,AK,ABILIFY MA,98.0,98.0,165146.32,165146.32,0.00


In [10]:
# read in drug info
druglist = pd.read_excel('20180208_HEDIS 2018_NDC_MLD_Directory_Complete_Workbook.xlsx',
                         'Medications List to NDC Codes')
druglist.head()

,Medication List,NDC Code,Brand Name,Generic Product Name,Route,Description,Drug ID,Drug Name,Package Size,Unit,Dose,Form,MED Conversion Factor
0,5-ARI Medications,54039513,Dutasteride,dutasteride 0.5 mg oral capsule,oral,5-ARI Medications,d04788,NaN,NaN,NaN,NaN,NaN,NaN
1,5-ARI Medications,54039522,Dutasteride,dutasteride 0.5 mg oral capsule,oral,5-ARI Medications,d04788,NaN,NaN,NaN,NaN,NaN,NaN
2,5-ARI Medications,93565556,Dutasteride,dutasteride 0.5 mg oral capsule,oral,5-ARI Medications,d04788,NaN,NaN,NaN,NaN,NaN,NaN
3,5-ARI Medications,93565598,Dutasteride,dutasteride 0.5 mg oral capsule,oral,5-ARI Medications,d04788,NaN,NaN,NaN,NaN,NaN,NaN
4,5-ARI Medications,115143808,Dutasteride,dutasteride 0.5 mg oral capsule,oral,5-ARI Medications,d04788,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# new dataframe with only opioid drugs
# we can tell if a drug is an opioid if the MED (Morphine Equivalent Dose) is not nan
opioiddrugs = druglist[druglist['MED Conversion Factor'].notnull()].copy()
opioiddrugs['Drug Name'] = opioiddrugs['Drug Name'].str.upper()
opioiddrugs['Brand Name'] = opioiddrugs['Brand Name'].str.upper()
opioiddrugs.head()

,Medication List,NDC Code,Brand Name,Generic Product Name,Route,Description,Drug ID,Drug Name,Package Size,Unit,Dose,Form,MED Conversion Factor
52293,Opioid Medications,591264101,APAP/BUTALBITAL/CAFFEINE/CODEINE,acetaminophen/butalbital/caffeine/codeine 300 ...,oral,Opioid Medications,d03425,CODEINE,NaN,mg,30.0,NaN,0.15
52294,Opioid Medications,52544008201,FIORICET WITH CODEINE,acetaminophen/butalbital/caffeine/codeine 300 ...,oral,Opioid Medications,d03425,CODEINE,NaN,mg,30.0,NaN,0.15
52295,Opioid Medications,143300001,APAP/BUTALBITAL/CAFFEINE/CODEINE,acetaminophen/butalbital/caffeine/codeine 325 ...,oral,Opioid Medications,d03425,CODEINE,NaN,mg,30.0,NaN,0.15
52296,Opioid Medications,247084530,FIORICET WITH CODEINE,acetaminophen/butalbital/caffeine/codeine 325 ...,oral,Opioid Medications,d03425,CODEINE,NaN,mg,30.0,NaN,0.15
52297,Opioid Medications,591322001,APAP/BUTALBITAL/CAFFEINE/CODEINE,acetaminophen/butalbital/caffeine/codeine 325 ...,oral,Opioid Medications,d03425,CODEINE,NaN,mg,30.0,NaN,0.15


In [12]:
opioiddrugs.columns

Index(['Medication List', 'NDC Code', 'Brand Name', 'Generic Product Name',
       'Route', 'Description', 'Drug ID', 'Drug Name', 'Package Size', 'Unit',
       'Dose', 'Form', 'MED Conversion Factor'],
      dtype='object')

In [13]:
opioiddrugs['Drug Name'].unique()

array(['CODEINE', 'DIHYDROCODEINE', 'HYDROCODONE', 'OXYCODONE',
       'PENTAZOCINE', 'TRAMADOL', 'OPIUM', 'BUPRENORPHINE', 'BUTORPHANOL',
       'FENTANYL', 'HYDROMORPHONE', 'LEVORPHANOL', 'MEPERIDINE',
       'METHADONE', 'MORPHINE', 'MORPHINE EQUIVALENT', 'OXYMORPHONE',
       'TAPENTADOL'], dtype=object)

In [14]:
## convert drug name column to list
opioids = opioiddrugs['Drug Name'].unique().tolist()
## join list of strings to create regex
opioids = '|'.join(opioids)
## use str.contains to match pattern
opioids_df = drugs2[drugs2['productname'].str.contains(opioids, na=False)]

In [15]:
opioids

'CODEINE|DIHYDROCODEINE|HYDROCODONE|OXYCODONE|PENTAZOCINE|TRAMADOL|OPIUM|BUPRENORPHINE|BUTORPHANOL|FENTANYL|HYDROMORPHONE|LEVORPHANOL|MEPERIDINE|METHADONE|MORPHINE|MORPHINE EQUIVALENT|OXYMORPHONE|TAPENTADOL'

In [16]:
# test to make sure we have all the opioid drugs
drugs2.loc[(drugs2.productname == 'HYDROCODON') | (drugs2.productname == 'HYDROMORP')].head()

,state,productname,unitsreimbursed,numberofprescriptions,totalamountreimbursed,medicaidamountreimbursed,nonmedicaidamountreimbursed
784,AK,HYDROCODON,1.952154e+06,39718.0,739899.67,738375.47,1524.20
3387,AL,HYDROCODON,1.000201e+07,195610.0,3950713.32,3295397.95,655315.37
6247,AR,HYDROCODON,8.757404e+06,147656.0,2755292.77,2745125.06,10167.71
8408,AZ,HYDROCODON,1.890895e+07,330257.0,5079746.82,5054637.71,25109.11
12067,CA,HYDROCODON,1.300894e+08,2365523.0,43016748.55,41496813.82,1519934.73


In [37]:
drugs2.loc[drugs2.productname.str.contains('TYLENOL') == True]

,state,productname,unitsreimbursed,numberofprescriptions,totalamountreimbursed,medicaidamountreimbursed,nonmedicaidamountreimbursed
5047,AL,TYLENOL WI,0.0,0.0,0.0,0.0,0.0
14447,CA,TYLENOL W/,0.0,0.0,0.0,0.0,0.0
23296,CT,TYLENOL W/,0.0,0.0,0.0,0.0,0.0
26499,DC,TYLENOL WI,0.0,0.0,0.0,0.0,0.0
30191,DE,TYLENOL WI,0.0,0.0,0.0,0.0,0.0
40589,HI,TYLENOL WI,0.0,0.0,0.0,0.0,0.0
46432,ID,TYLENOL-CO,0.0,0.0,0.0,0.0,0.0
50795,IL,TYLENOL W/,0.0,0.0,0.0,0.0,0.0
63349,KY,TYLENOL-CO,0.0,0.0,0.0,0.0,0.0
70086,MA,TYLENOL WI,0.0,0.0,0.0,0.0,0.0


In [17]:
# it looks ike hydrocodone and hydromorphone didn't match on the opioid string
# adding our hydrocodone/hydromorphone drugs
hydros = drugs2[drugs2['productname'].str.contains('HYDROCOD', na=False)]
hydroms = drugs2[drugs2['productname'].str.contains('HYDROMOR', na=False)]

In [18]:
opioids_df = opioids_df.append(hydros)
opioids_df = opioids_df.append(hydroms)

In [19]:
opioids_df.loc[(opioids_df.productname.str.contains('HYDROCODON')) | (opioids_df.productname.str.contains('HYDROMOR'))].sample(10)

,state,productname,unitsreimbursed,numberofprescriptions,totalamountreimbursed,medicaidamountreimbursed,nonmedicaidamountreimbursed
39262,HI,HYDROCODON,3.054930e+06,58002.0,11187950.83,11187464.70,486.13
121755,OH,HYDROMORPH,8.310270e+05,50733.0,286004.55,280183.01,5821.54
135361,PA,HYDROMORPH,8.306040e+05,35285.0,363925.86,352220.62,11705.24
48584,IL,HYDROCODON,4.322964e+07,711085.0,10578366.26,10500715.64,77650.62
80651,MN,HYDROCODON,8.558088e+06,171068.0,2504954.00,2496040.46,8913.54
149791,TN,HYDROMORPH,1.395760e+05,3156.0,22580.92,22547.02,33.90
61847,KY,HYDROMORPH,2.894650e+05,17198.0,140798.33,126831.56,13966.77
89229,MT,HYDROMORPH,2.462015e+05,6961.0,140891.17,140382.30,508.87
72063,MD,HYDROCODON,4.316340e+06,98733.0,936867.31,913481.28,23386.03
20959,CT,HYDROCODON,3.379893e+06,69634.0,1323091.48,1312249.99,10841.49


In [20]:
# checking to make sure we sorted out all the opioids ..which we're missing brand names
opioids_df.productname.unique()

array(['CODEINE SU', 'FENTANYL', 'FENTANYL C', 'MEPERIDINE', 'METHADONE',
       'MORPHINE S', 'OPIUM TINC', 'OXYCODONE', 'OXYCODONE-',
       'TRAMADOL H', 'CODEINE PH', 'FENTANYL M', 'FENTANYL P',
       'FENTANYL R', 'FENTANYL T', 'MORPHINE P', 'OXYCODONE/',
       'TRAMADOL 5', 'TRAMADOL A', 'TRAMADOL T', 'CODEINE-GU',
       'FENTANYL 0', 'FENTANYL 1', 'FENTANYL 2', 'FENTANYL 3',
       'FENTANYL 5', 'FENTANYL 6', 'FENTANYL 7', 'FENTANYL 8',
       'TRAMADOL E', 'TRAMADOL-A', 'MORPHINE 1', 'MORPHINE 2',
       'MORPHINE 4', 'MORPHINE 8', 'TRAMADOL/A', 'MORPHINE 5',
       'MORPHINE 3', 'MORPHINE 0', 'TRAMADOL W', 'GG/CODEINE', 'OPIUM',
       'HYDROCODON', 'HYDROCOD/A', 'HYDROCOD-H', 'DIHYDROCOD',
       'HYDROCOD/I', 'HYDROCOD-C', 'HYDROMORPH', 'HYDROMOREP'],
      dtype=object)

In [22]:
opioiddrugs['Brand Name'].unique()

array(['APAP/BUTALBITAL/CAFFEINE/CODEINE', 'FIORICET WITH CODEINE',
       'APAP/CAFFEINE/DIHYDROCODEINE BITARTRATE', 'TREZIX',
       'ACETAMINOPHEN-CODEINE PHOSPHATE',
       'CAPITAL AND CODEINE SUSPENSION', 'TYLENOL WITH CODEINE #3',
       'TYLENOL WITH CODEINE #4', 'ACETAMINOPHEN-HYDROCODONE BITARTRATE',
       'VICODIN HP', 'XODOL', 'ZOLVIT', 'LORTAB ELIXIR', 'LORTAB',
       'VICODIN', 'VICODIN ES', 'NORCO', 'LORTAB 10/325', 'LORCET HD',
       'ZAMICET', 'VERDROCET', 'LORTAB 5/325', 'LORCET', 'LORTAB 7.5/325',
       'LORCET PLUS', 'HYCET', 'PRIMLEV',
       'ACETAMINOPHEN-OXYCODONE HYDROCHLORIDE', 'PERCOCET 10/325',
       'ENDOCET 10/325', 'ENDOCET 2.5/325', 'PERCOCET 2.5/325', 'ROXICET',
       'PERCOCET 5/325', 'ENDOCET 5/325', 'ENDOCET 7.5/325',
       'PERCOCET 7.5/325', 'XARTEMIS XR', 'ACETAMINOPHEN-PENTAZOCINE',
       'ACETAMINOPHEN-TRAMADOL HYDROCHLORIDE', 'ULTRACET',
       'ASPIRIN/BUTALBITAL/CAFFEINE/CODEINE', 'ASCOMP WITH CODEINE',
       'FIORINAL WITH CODEINE',

In [26]:
# repeat filter with brand names
## convert drug name column to list
opioidbrands = opioiddrugs['Brand Name'].unique().tolist()
## join list of strings to create regex
opioidbrands = '|'.join(opioidbrands)
## use str.contains to match pattern
opioidbrands_df = drugs2[drugs2['productname'].str.contains(opioids, na=False)]

In [27]:
opioidbrands

'APAP/BUTALBITAL/CAFFEINE/CODEINE|FIORICET WITH CODEINE|APAP/CAFFEINE/DIHYDROCODEINE BITARTRATE|TREZIX|ACETAMINOPHEN-CODEINE PHOSPHATE|CAPITAL AND CODEINE SUSPENSION|TYLENOL WITH CODEINE #3|TYLENOL WITH CODEINE #4|ACETAMINOPHEN-HYDROCODONE BITARTRATE|VICODIN HP|XODOL|ZOLVIT|LORTAB ELIXIR|LORTAB|VICODIN|VICODIN ES|NORCO|LORTAB 10/325|LORCET HD|ZAMICET|VERDROCET|LORTAB 5/325|LORCET|LORTAB 7.5/325|LORCET PLUS|HYCET|PRIMLEV|ACETAMINOPHEN-OXYCODONE HYDROCHLORIDE|PERCOCET 10/325|ENDOCET 10/325|ENDOCET 2.5/325|PERCOCET 2.5/325|ROXICET|PERCOCET 5/325|ENDOCET 5/325|ENDOCET 7.5/325|PERCOCET 7.5/325|XARTEMIS XR|ACETAMINOPHEN-PENTAZOCINE|ACETAMINOPHEN-TRAMADOL HYDROCHLORIDE|ULTRACET|ASPIRIN/BUTALBITAL/CAFFEINE/CODEINE|ASCOMP WITH CODEINE|FIORINAL WITH CODEINE|SYNALGOS-DC|ASPIRIN/CAFFEINE/DIHYDROCODEINE|ASA/CARISOPRODOL/CODEINE PHOSPHATE|ASPIRIN-OXYCODONE|ENDODAN|BELLADONNA ALKALOIDS-OPIUM|BUPRENORPHINE|BUTRANS|BELBUCA|BUNAVAIL|BUTORPHANOL TARTRATE|CODEINE SULFATE|FENTORA|SUBSYS|ABSTRAL|FENTANYL|FE

In [43]:
opioidbrands = 'BUTALBITAL|FIORICET|DIHYDROCODEINE|TREZIX|CODEINE|TYLENOL|VICODIN|XODOL|ZOLVIT|NORCO|LORTAB|LORCET|ZAMICET|VERDROCET|HYCET|PRIMLEV|PERCOCET|ENDOCET|ROXICET|XARTEMIS|PENTAZOCINE|ULTRACET|ASCOMP|FIORINAL|SYNALGOS|ENDODAN|BELLADONNA|BUPRENORPHINE|BUTRANS|BELBUCA|BUNAVAIL|BUTORPHANOL|FENTORA|SUBSYS|ABSTRAL|FENTANYL|FENTANYLDURAGESIC|LAZANDA|ACTIQ|DURAGESIC-25|IONSYS|ZOHYDRO ER|HYSINGLA|REPREXAIN|IBUDONE|XYLON|VICOPROFEN|HYDROMORPHONE|DILAUDID|EXALGO|LEVORPHANOL|DEMEROL|MEPERIDINE|DOLOPHINE|METHADONE|METHADOSE|MORPHINE SULFATE|KADIAN|MS CONTIN|MORPHABOND|AVINZA|ARYMO|EMBEDA|PENTAZOCINE|PAREGORIC|OPIUM|OXYCONTIN|XTAMPZA ER|ROXICODONE|OXYFAST|OXECTA|OXAYDO|OPANA|NUCYNTA|CONZIP|RYZOLT|TRAMADOL|ULTRAM|RYBIX'

In [44]:
brands_df = drugs2[drugs2['productname'].str.contains(opioidbrands, na=False)]

In [45]:
brands_df

,state,productname,unitsreimbursed,numberofprescriptions,totalamountreimbursed,medicaidamountreimbursed,nonmedicaidamountreimbursed
145,AK,ASCOMP WIT,0.000,0.0,0.00,0.00,0.00
192,AK,BELLADONNA,0.000,0.0,0.00,0.00,0.00
242,AK,BUTALBITAL,29494.000,655.0,43304.54,42695.12,609.42
244,AK,BUTRANS,861.000,221.0,79920.82,79361.86,558.96
366,AK,CODEINE SU,0.000,0.0,0.00,0.00,0.00
437,AK,DEMEROL,59.375,59.0,1180.92,1045.40,135.52
481,AK,DILAUDID,0.000,0.0,0.00,0.00,0.00
544,AK,EMBEDA,0.000,0.0,0.00,0.00,0.00
555,AK,ENDOCET,10571.000,187.0,7306.41,7306.41,0.00
610,AK,EXALGO,0.000,0.0,0.00,0.00,0.00


In [46]:
# appending the brand name dataframe to generics
opioids_df = opioids_df.append(brands_df)
opioids_df.sample(10)

,state,productname,unitsreimbursed,numberofprescriptions,totalamountreimbursed,medicaidamountreimbursed,nonmedicaidamountreimbursed
93178,NC,TYLENOL-CO,0.0,0.0,0.00,0.00,0.00
9805,AZ,VICODIN,33211.0,1041.0,50025.14,49899.42,125.72
151614,TX,BUTRANS 5,5141.0,1285.0,281601.86,280787.04,814.82
59098,KS,OPANA ER,3540.0,63.0,42904.91,42778.91,126.00
108086,NM,HYDROMORPH,222712.5,12457.0,73724.00,73668.95,55.05
75466,ME,METHADONE,773405.0,6156.0,124501.89,124148.34,353.55
8423,AZ,HYSINGLA E,0.0,0.0,0.00,0.00,0.00
57089,KS,CODEINE-GU,57937.0,426.0,5645.18,5084.74,560.44
20437,CT,EXALGO,0.0,0.0,0.00,0.00,0.00
143627,SC,CODEINE SU,0.0,0.0,0.00,0.00,0.00


In [47]:
opioids_df.shape

(3612, 7)

In [48]:
# saving new opioid dataframe to csv
opioids_df.to_csv('medicaid_opioids_2016.csv')